In [1]:
import pickle
#from utils import * 
import pandas as pd
import numpy as np
import json
import nltk
#import seaborn as sns
#import matplotlib.pyplot as plt

In [3]:
def json_df(datapass):
    '''
    Load the json file and parse the file to pandas dataframe format
    
    Input:
        datapass(str) : directory to the json file
    Output:
        df(dataframe) : pandas dataframe object
    '''
    
    data = [] 
    with open(datapass, 'r', encoding='utf-8') as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

#business and review json files --> dataframes

DATAPASS1 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\business.json'
DATAPASS2 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\review.json'

#business = json_df(DATAPASS1)
reviews = json_df(DATAPASS2)

In [15]:
def load_topic_model(vectorizer_file_name, topic_model_file_name):
    with open(vectorizer_file_name, "rb") as f:
        vectorizer = pickle.load(f)
    with open(topic_model_file_name, "rb") as f:
        topic_model = pickle.load(f)
    return vectorizer, topic_model

#pizza

path1 = 'C:\\Users\\cdchang\\Desktop\\yelp-topic-model\\model\\pizza_tfidf_vectorizer.pkl'
path2 = 'C:\\Users\\cdchang\\Desktop\\yelp-topic-model\\model\\pizza_nmf.pkl'
path3 = 'C:\\Users\\cdchang\\Desktop\\yelp-topic-model\\model\\pizza_count_vectorizer.pkl'
path4 = 'C:\\Users\\cdchang\\Desktop\\yelp-topic-model\\model\\pizza_lda.pkl'
vec, tm = load_topic_model(path1, path2)
vec2, tm2 = load_topic_model(path3, path4)

In [5]:
#k index embedding method
def get_avg_k_index_for_doc(doc, vec, tm):
    sent_text = nltk.sent_tokenize(doc)
    if (int(len(sent_text)) == 0):
        return []
    else:
        tm_mat = tm.comments
        k_index = np.array([0.]*tm_mat.shape[0])
        for s in sent_text:
            k, p = get_k_index(s, vec, tm)
            k_index += p
        return k_index/int(len(sent_text))

#use this 
def business_id_retrieval(cat, business):
    '''
    Input:
        cat(str) : category
        business(dataframe) : the business data
    Output:
        id_list(set) : business ids of a particular category
    '''
    id_list = set()
    idx = 0
    for row in business.values:
        categories = row[3]
        if cat in categories:
            id_list.add(row[2])
    return id_list

business_id_retrieval('Shopping', business)

#get_avg_k_index_for_doc("uYHaNptLzDLoV_JZ_MuzUA",vec, tm)

{'LRUsIQmzycZHj3YLzcnVGA',
 '1xExbYcT2SnnjyPXV5D0xQ',
 '5f3SBBaVoeI4tPgx_MTRHg',
 'vuvnexwJzb1ydKLacBP8xg',
 'DegZrwS3r5mE2oLXbg08Hw',
 '1E_usKkNL1N_--frAnkdBA',
 'co9I6wlR0YhP0THsZBGafg',
 'wbfzo656xbGiiGl5lIt23w',
 'wQEqHgOkgIv87HxZ8WQg1Q',
 'Vxg55XMENoMIUCYVTQUa1Q',
 'CmbBOsZqXGkaFjJXfKz3CA',
 '9LwvVV5aU7KK4RpjrI-sYA',
 'BhCAVw284KrfM6XQVAxv1A',
 'GoWSjWxoeDbcyPSfkxt8-w',
 'waw-t98gJ4ppxOW1xApk1Q',
 'zx_xDVaVQlE3eDoFFENkow',
 '9Ky77tqqzL6Fv9wX9V9vlQ',
 'NETsamdEjJADcoM-SXpskA',
 'nfg1c2ECbnbNWM5gt0LYiA',
 '-Mwyd0NjrJ6hS0LzRlOLTg',
 'j65zwVK-QJQ78gAkZ9smZw',
 '2A-bNVndawMfIuNNXfJzfg',
 '5pKS-o4Z96wYbC8YSZeTEw',
 '_UQf5ZgrVbk7aRDHvBSMmw',
 '8PkISUNAHvIRMSRDdMawZw',
 'fiFk4W6rYE2CddBHY58Zpg',
 'Dy3gsxycAonPploaY3xHcA',
 'R_QmwBJhwI5K0o6WZRTttg',
 '7rrrJqeVdtpUufziEPLEyw',
 'gZI007iLBe_LZbrHSDxHtg',
 '04Ap824N__YXYNvaVmDYeQ',
 'QFYJHmCZlTJ7wX2syK0LlQ',
 'GQSw16RWrdZpy7zqnaq2Bg',
 'wD7rrbL1btbK_yyyPuSTfg',
 'yIEUQ1MCbvDuGID1h9DDUA',
 'TsJb9kyNY6wCa8V14sEj9A',
 'MQn8y3umLiePBcCeokn3eQ',
 

In [28]:
#WORK IN PROGRESS
#subset of pizza / chinese reviews (obtain random sample)
def generate_k_index_trend(b_id, reviews):
    #idlist = business_id_retrieval(cat, business) for later
    s =reviews.loc[reviews['business_id'] == "uYHaNptLzDLoV_JZ_MuzUA"]
    if s['date'][0][0:4] 
    #for i in idlist:
        #if i in reviews.values("business_id"):       
    #reviews.loc[reviews['business_id'] == b_id]

generate_k_index_trend('Shopping', reviews)

In [25]:
#ultimately: find subset of pizza / chinese restaurants 

#for a single business_id
def examine_reviews(b_id, reviews):
    '''
    concatenate a particular business's review text with the corresponding dates for time series analysis
    
    Input: specific business id, review json file
    Output:
    '''
    subset = reviews.loc[reviews['business_id'] == b_id]
    date = subset.loc[:, 'date']
    text = subset.loc[:, 'text']
    return pd.concat([date, text], axis=1, join='inner')


examine_reviews("uYHaNptLzDLoV_JZ_MuzUA", reviews)

,date,text
0,2016-07-12,My girlfriend and I stayed here for 3 nights a...
1,2016-10-02,If you need an inexpensive place to stay for a...
2,2015-09-17,Mittlerweile gibt es in Edinburgh zwei Ableger...
3,2016-08-21,Location is everything and this hotel has it! ...
4,2013-11-20,gute lage im stadtzentrum. shoppingmeile und s...
5,2016-06-05,Erstklassige Lage. Dazu ist alles geschrieben ...
6,2015-02-21,"Beautiful space, great location, staff rock. T..."
7,2013-07-07,This is a fairly new property I think. It is a...
8,2013-04-27,First time at this group of hotels. Pretty new...
9,2015-04-13,Location location location! \n\nMotel One is j...


In [ ]:
#this works for extracting date
#final_dates=[]
#dates = reviews.date.where(reviews.business_id == "uYHaNptLzDLoV_JZ_MuzUA").dropna()
#for d in range(len(dates)):
#    yr = pd.to_numeric(subset['date'][d][0:4])
#    final_dates.append(yr)
    #final_dates.append((str(d)[0:4]))
    
#final_dates